## Computer Vision의 활용분야는 다음과 같다.

- Classification(분류) : Input에 대해 하나의 label을 예측
    - AlexNet
    - ResNet
    - Xception


- Semantic Segmentation(분할) : 모든 pixel의 label 예측
    - FCN
    - SegNet
    - DeepLab


- detection(발견) : 물체의 label을 예측하면서 그 물체가 어디있는지 정보를 제공
    - YOLO
    - R-CNN

# Semantic Segmentation


<br>
<br>
<img src="../images/vggfcl.png" width=500>


- AlexNet, VGG 같이 분류에 쓰이는 모델은 Semantic Segmentation 을 하는데 적합하지 않다. parameter 의 개수와 차원을 줄이는 layer 를 가지고 있어 Fully Connected Layer에 의해 위치 정보를 잃게 된다

<img src="../images/DU.png">

- 그래서 Semantic Segmentation 모델들은 보통 Downsampling & Upsampling 의 형태를 가진다.


- **Downsampling** : 차원을 줄이는 것이 목적(stride 2 이상의 Convolution이나 pooling을 사용)


- 마지막에 Fully-Connected Layer를 넣지 않고, Fully Connected Network 를 주로 사용


- **Upsampling** : Downsampling 을 통해서 받은 결과의 차원을 늘려서 인풋과 같은 차원으로 만들어 줌. 주로 Strided Transpose Convolution 을 사용

### Fully Convolutional Network(FCN)

<img src="../images/fcn.png" width=600>

- 기본적으로 VGG 모델 base에서 

- 2014년 이 논문이 나온 후 Fully Connected layer가 없는 cnn이 통용되었고, 이후 대부분의 Segmentation은 FCN은 기반으로 하였다


- Semantic Segmentation를 하기위한 핵심아이디어이다.


- Transposed Convolution을 통해서 낮은 해상도의 heap map 을 Upsampling 해서 input과 같은 크기의 맵을 만든다

- 마지막 output을 dense에서 conv로 바꾼다 $\rightarrow$convolutionalization이라고 한다.


### Deconvolution(conv transpose)

input, kernel, padding, stride = $i, k, p, s$라고 할 때

$$ \text{Down sampling} : o = \frac{i+2p-k}{s+1}  $$



$$ \text{Up sampling} : {(i-1)*s+k}-2p $$


- Upsampling 하는 것을 Deconvolution이라 한다.

- convolution의 역 연산

- 사실 역으로 연산하는 것은 불가능하다. 14라는 숫자로 2x7을 복원 할 수는 없다 대략적인 크기 복원

- 하지만 물체의 크기를 잘 인식못하거나 해상도를 복원하는 과정해서 결과가 정확하지 않은 문제 발생

---
# Detection

Classification(분류)와 물체의 위치를 포함한 Bounding box를 찾는  Localization을 모두 포함
### R-CNN : Regions with Convolutional Neuron Networks features

1. 보통 2000개의 regions를 뽑는다.
2. 같은 크기로 맞춘다.
3. features는 AlexNet사용
4. SVMs로 분류한다. (가장 단순한 방법) 2000개를 뽑으면 2000번의 연산을 해야한다는 단점이 있다.


### SPPNet

- R-CNN의 단점 : image에 2000개의 bounding-box를 뽑으면 cnn을 모두 통과시켜야함.(시간이 오래걸림)
- 이미지에서 bounding-box를 뽑고 해당 위치의 tensor만 가져와 한번만 통과시킴.


### Fast R-CNN

1. Takes an input and a set of bounding boxes. 이미지에서 2000개를 뽑는다.
2. Generated convolutional feature map. CNN한번 통과
3. For each region, get a fixed length feature from ROI pooling.  각각의 Region에서 length feature from ROI pooling뽑음
4. Two outputs: class and bounding-box regressor. 그 후 뉴럴네트워크를 통해 bounding-box 를 어떻게 움직일지 결정(SPPNet과의 차이점)


### Region Proposal Network(RPN)

- image에서 특정 영역의 bounding-box에 물체가 있을지 판단한다
- 무슨 물체인지는 모르지만 bounding box를 찾으면 Faster R-CNN으로 보냄.
- 마찬가지로 FCN을 사용(물체정보를 FCN이 담는다.)


- 9 : Three different region sizes(128,256,512) with three different ratios(1:1, 1:2, 2:1) 3가지 size 마다 3가지 비율의 9가지 정보
- 4 : four bounding box regression parameters, 위 정보의 bounding box를 키울지,줄일지와 x,y offset의 4가지 정보
- 2: box classification(whether to use it or not) 해당 bounding box가 유효한지 아닌지 판단
총 9*(4+2)의 54가지 영역

### Faster R-CNN

- Region Proposal Network(RPN)도 학습하자는 아이디어
- 해당 Region의 유효정보를 담은 RPN을 Fast R-CNN에 더해준다.

`Faster R-CNN = Region Proposal Network(RPN) + Fast R-CNN`

### YOLO

- YOLO is an extremely fast object detection algorithm:
- YOLO는 Faster R-CNN보다 훨씬 빠르다.


- It simultaneously predicts multiple bounding boxes and class probabilities.(No explicit bounding box sampling compared with Faster R-CNN)
- RPN을 거치지않고 한번에 bounding box를 얻는 방식이다.


- image가 들어오면 SxS grid로 나눈다. 
- image안의 찾고싶은 물체 중간이 해당 그리드 안에 들어오면 해당 물체의 bounding box와 해당 물체를 동시에 찾는다.

B\*5 x,y,w,h,유효(사용할지말지) 5가지 로 판단
C -> Number of classes
YOLO의 Tensor 는 SxSx(B\*5+C) 이다.
 
- 즉 YOLO는 Fatser R-CNN에 비해 bounding box를 빨리 찾고 동시에 물체를 찾으므로 더 빠르다.

[yolo 원리](https://ctkim.tistory.com/91)

### 2020년 이후
2017년 6월 Google이 NLP분야 모델인 Transformer 모델 소개  


2020년 3월 Goggle이 NLP분야에서 사용하던 Transformer 기술을 CV에 적용시도  

2020년 10월 ViT 모델(Vision Transformer) 등장, object detection 모델의 backbone을 cnn에서 Transformer로 바꾸려는 시도
- 하지만 ViT 모델은 지나치게 커서 방대한 양의 데이터를 필요로한다


2020 12월 Facebook이 이 단점을 보완한 DeiT 모델 제안
- 효율적 학습을 위해 Knowledge distillation 기법을 transformer 모델에 적용
- 방대한 데이터가 없어도 성능이 좋은 모델 선보임
2021년 object 모델의 backbone을 ViT 모델을 사용하는 연구가 진행


2021년 DeiT 기반의 YOLOS(기존의 YOLO 모델과 전혀관계 없음)모델을 만드나 좋은 성과는 못 거둠


**2021년 3월 Microsoft에서 새로운 ViT 모델인 Swin Transformer 모델 선보임**
<img src="../images/swim.png" width=500>

- Segmentation과 Transformer 모두 계산량이 많기 때문에 같이 접목하면 문제가생긴다.
- 이를 극복하기 위해 hierarchical feature map과 Shifted window (Swin) block 기술 사용
- Swin Transformer의 경우 작은 패치에서 시작해서 주변과 merging한다
- 이 때 window 단위로 self attention을 진행
   
<img src="../images/window.png" width=500>

- 핵심기술인 shifted window 
-  4개로 나눠진 window 단위로 self attention을 진행한다. 하지만 이대로는 나눠진 window 끼리 connection을 표현 할 수 없습니다.
- 그래서 shifted window 방법을 제안, window를 window_size//2 만큼 우측 하단으로 이동시켜 window를 새로 만듬.

그 이후 Soft Teacher+Swin-L 모델이 나와 paperwithcode의 object detection 순위는 모두 swin transformer이다.  
[Detection 대회 순위](https://paperswithcode.com/sota/object-detection-on-coco?p=swin-transformer-hierarchical-vision)